In [21]:
using DataFrames
using CSV
using LinearAlgebra

struct Path
    p::Union{Nothing, Vector{Int}}
    d::Float64
end

function Base.:+(a::Path, b::Path)
    isnothing(a.p) && return b
    isnothing(b.p) && return a
    return a.d < b.d ? a : b
end

function Base.:*(a::Path, b::Path)
    isnothing(a.p) && return Path(nothing, Inf)
    isnothing(b.p) && return Path(nothing, Inf)
    @assert last(a.p) == first(b.p)
    return Path([a.p; b.p[2:end]], a.d + b.d)
end

Base.zero(::Type{Path}) = Path(nothing, Inf)

# mostra bonitinho
Base.show(io::IO, p::Path) = print(io, p.p === nothing ? "Nothing" : join(p.p, "->"))

# igualdade
Base.:(==)(a::Path, b::Path) = a.p == b.p && a.d == b.d

# matriz identidade
function Identity(dim, tipo)
    return [Path(i == j ? [i] : nothing, i == j ? 0.0 : Inf) for i in 1:dim, j in 1:dim]
end

# multiplicacao de matriz customizada para o tipo Path
function Base.:*(A::Matrix{Path}, B::Matrix{Path})
    m, n = size(A)
    p = size(B, 2)
    result = Matrix{Path}(undef, m, p)
    
    for i in 1:m
        for j in 1:p
            paths = [A[i,k] * B[k,j] for k in 1:n]
            result[i,j] = reduce(+, filter(path -> path.p !== nothing, paths), init=Path(nothing, Inf))
        end
    end
    
    return result
end

df = CSV.read("dist_brasil_clean.csv", DataFrame)

# fronteiras entre estados
fronteiras = [
    (1100205, 5103403), (1100205, 1200401), (1100205, 1302603),
    (1200401, 1302603), (1302603, 1400100), (1302603, 5103403),
    (1501402, 5103403), (1501402, 1400100), (1501402, 1600303),
    (1501402, 2111300), (1501402, 1721000), (1721000, 2111300),
    (1721000, 2211001), (1721000, 2927408), (1721000, 5208707),
    (5103403, 5002704), (5103403, 5208707), (5103404, 1721000),
    (5208707, 5300108), (5208707, 2927408), (5208707, 5002704),
    (5208707, 3106200), (5002704, 3106200), (5002704, 3550308),
    (5002704, 4106902), (4106902, 4205407), (4106902, 3550308),
    (4205407, 4314902), (3550308, 3304557), (3550308, 3106200),
    (3304557, 3106200), (3304557, 3205309), (3205309, 3106200),
    (3205309, 2927408), (3106200, 2927408), (2927408, 2111300),
    (2927408, 2211001), (2927408, 2800308), (2927408, 2704302),
    (2927408, 2611606), (2800308, 2704302), (2704302, 2611606),
    (2611606, 2211001), (2611606, 2304400), (2611606, 2507507),
    (2507507, 2408102), (2507507, 2304400), (2408102, 2304400),
    (2304400, 2211001), (2211001, 2111300)
]

# filtra o dataframe para incluir apenas as fronteiras
df_fronteiras = filter(row -> (row.orig, row.dest) in fronteiras || (row.dest, row.orig) in fronteiras, df)

# cria lista com todas as capitais envolvidas no processo
capitais = unique(vcat(df_fronteiras.orig, df_fronteiras.dest))

# cria um dicionario que mapeia capitais para indices
capitais_dict = Dict(capital => i for (i, capital) in enumerate(capitais))

dicionario_cidade_codigo = Dict(
    "Rio Branco" => 1200401,
    "Maceió" => 2704302,
    "Macapá" => 1600303,
    "Manaus" => 1302603,
    "Salvador" => 2927408,
    "Fortaleza" => 2304400,
    "Brasília" => 5300108,
    "Vitória" => 3205309,
    "Goiânia" => 5208707,
    "São Luís" => 2111300,
    "Cuiabá" => 5103403,
    "Campo Grande" => 5002704,
    "Belo Horizonte" => 3106200,
    "Belém" => 1501402,
    "João Pessoa" => 2507507,
    "Curitiba" => 4106902,
    "Recife" => 2611606,
    "Teresina" => 2211001,
    "Rio de Janeiro" => 3304557,
    "Natal" => 2408102,
    "Porto Alegre" => 4314902,
    "Porto Velho" => 1100205,
    "Boa Vista" => 1400100,
    "Florianópolis" => 4205407,
    "São Paulo" => 3550308,
    "Aracaju" => 2800308,
    "Palmas" => 1721000
)

n = length(capitais)

matriz_adj = Identity(n, Path)

for row in eachrow(df_fronteiras)
    i = capitais_dict[row.orig]
    j = capitais_dict[row.dest]
    matriz_adj[i, j] = Path([i, j], row.dist)
    matriz_adj[j, i] = Path([j, i], row.dist)
end

function menor_caminho(matriz_adj, origem, destino)
    n = size(matriz_adj, 1)
    pm = copy(matriz_adj)
    fixed_points = 0

    while fixed_points < 2
        pm_old = copy(pm)
        pm = pm * pm
        if pm == pm_old
            fixed_points += 1
        else
            fixed_points = 0
        end
    end

    return pm[origem, destino]
end


println("Nome da cidade e seu índice:")
for (capital, index) in capitais_dict
    cidade = first(key for key in keys(dicionario_cidade_codigo) if dicionario_cidade_codigo[key] == capital)
    println("$cidade: $index")
end

Nome da cidade e seu índice:
Belém: 5
Palmas: 6
João Pessoa: 11
Teresina: 8
Maceió: 13
Vitória: 17
Cuiabá: 23
Goiânia: 24
Curitiba: 20
Macapá: 25
Natal: 10
Porto Alegre: 26
Rio Branco: 2
Salvador: 15
Fortaleza: 9
Campo Grande: 22
Aracaju: 14
Manaus: 3
São Luís: 7
Rio de Janeiro: 18
Brasília: 27
Recife: 12
Porto Velho: 1
Belo Horizonte: 16
Boa Vista: 4
São Paulo: 19
Florianópolis: 21


In [22]:
println("Digite o índice da cidade de partida:")
origem_idx = parse(Int, readline())
println("Digite o índice da cidade de destino:")
destino_idx = parse(Int, readline())

caminho = menor_caminho(matriz_adj, origem_idx, destino_idx)
println("O menor caminho entre $origem_idx e $destino_idx é $caminho")

Digite o índice da cidade de partida:


Digite o índice da cidade de destino:
O menor caminho entre 1 e 14 é 1->23->24->15->14


In [23]:
#=using CSV, DataFrames

# Carregar o arquivo CSV
file_path = "dist_brasil.csv"  # Atualize o caminho do arquivo, se necessário
df = CSV.read(file_path, DataFrame; delim=';', normalizenames=true)

# Remover linhas que contenham "NA" na coluna 'dist'
clean_df = filter(row -> row.dist != "NA", df)

# Converter a coluna 'dist' para Float64
clean_df.dist = parse.(Float64, clean_df.dist)

# Salvar o DataFrame limpo em um novo arquivo (opcional)
output_path = "dist_brasil_clean.csv"
CSV.write(output_path, clean_df)

clean_df = select!(clean_df, Not(:dur))=#

In [24]:
#=using SimpleWeightedGraphs, Graphs, LinearAlgebra

unique_nodes = unique(vcat(clean_df[:, :orig], clean_df[:, :dest]))
num_nodes = length(unique_nodes)

dict = Dict(zip(unique_nodes, 1:num_nodes))=#

In [25]:
#=using LinearAlgebra

MatrizDistancia = Identity(num_nodes, Path)

for row in eachrow(clean_df)
    i, j = dict[row.orig], dict[row.dest]
    MatrizDistancia[i, j] = Path([i, j])
end

MatrizDistancia=#

In [26]:
#=function matrix_power(A::Matrix{Path}, k::Int)
    n = size(A, 1)
    result = Identity(n, Path)
    base = A

    while k > 0
        if k % 2 == 1
            result = mul!(result, result, base)
        end
        base = mul!(base, base, base)
        k ÷= 2
    end

    return result
end

pm = MatrizDistancia
n = 1
fixed_points = 0

while fixed_points < 3
    pm_old = copy(pm)
    pm = matrix_power(pm, 2)
    if pm == pm_old
        fixed_points += 1
    else
        fixed_points = 0
    end
    n += 1
end

println("O valor de n pelo método do ponto fixo é: ", n)=#